In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os.path

import torch as t
import pandas as pd
from PIL import Image
from daam import trace
import numpy as np
import torch.nn.functional as F
from matplotlib import pyplot as plt

from utils import utils
from utils.itdiffusion import DiffusionModel  # Info theoretic diffusion library and flow sampler
from utils.stablediffusion import StableDiffuser

/root/anaconda3/envs/infodp/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/root/anaconda3/envs/infodp/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


## Preprocess lora weight

In [25]:
# from safetensors.torch import load_file, save_file

# model_file = load_file("/root/tmp/EE142/model/Burberry.safetensors")

In [38]:
# new_dict = {}

# for k, v in model_file.items():
#     if "unet" in k:
#         # if "down" in k:
#         #     if v.shape == t.Size([128, 320, 1, 1]):
#         #         v = v.reshape(128, 320)
#         #         # print(k, v.shape)
#         #     elif v.shape == t.Size([128, 640, 1, 1]):
#         #         v = v.reshape(128, 640)
#         #     elif v.shape == t.Size([128, 1280, 1, 1]):
#         #         v = v.reshape(128, 1280)
#         #         # print(k, v.shape)

#         # if "up" in k:
#         #     if v.shape == t.Size([320, 128, 1, 1]):
#         #         v = v.reshape(320, 128)
#         #         # print(k, v.shape)
#         #     elif v.shape == t.Size([640, 128, 1, 1]):
#         #         v = v.reshape(640, 128)
#         #     elif v.shape == t.Size([1280, 128, 1, 1]):
#         #         v = v.reshape(1280, 128)
#         #         # print(k, v.shape)

#         new_dict[k] = v

In [39]:
# save_file(new_dict, "/root/tmp/EE142/model/Burberry_unet.safetensors")

## Info Decomp

In [33]:
# set hyper-parameters
seed = 42
data_in_dir = "/root/tmp/EE142/data/coco/val2017"
res_out_dir = "/root/tmp/EE142/data/results/test"
n_samples_per_point = 120
batch_size = 120
num_steps = 100
sdm_version = 'sdm_2_0_base' 
clip = 3

t.manual_seed(seed);

In [40]:
# load diffusion models
if sdm_version == 'sdm_2_0_base':
    sdm = StableDiffuser("/root/tmp/EE142/model/models--stabilityai--stable-diffusion-2-base/snapshots/fa386bb446685d8ad8a8f06e732a66ad10be6f47")
elif sdm_version == 'sdm_2_1_base':
    sdm = StableDiffuser("stabilityai/stable-diffusion-2-1-base")

logsnr_max, logsnr_min = sdm.logsnr_max, sdm.logsnr_min
logsnr_loc = logsnr_min + 0.5 * (logsnr_max - logsnr_min)
logsnr_scale = (1. / (2. * clip)) * (logsnr_max - logsnr_min)

latent_shape = (sdm.channels, sdm.width, sdm.height)
itd = DiffusionModel(sdm.unet, latent_shape, logsnr_loc=logsnr_loc, logsnr_scale=logsnr_scale, clip=clip,
                        logsnr2t=sdm.logsnr2t).to(sdm.device)

# Defines range of sigma/snr to use during sampling, based on training

sigma_min, sigma_max = utils.logsnr2sigma(logsnr_max), utils.logsnr2sigma(logsnr_min)

# Set schedule in Karras et al terms, "sigmas", where z = x + sigma epsilon.
schedule = utils.get_sigmas_karras(num_steps, sigma_min, sigma_max, device=itd.device)
# For generation, use schedule. For reversible sampling use the following, which
# doesn't go all the way to the limit sigma=0, snr=inf We can't approx score there so can't reverse
schedule_reversible = schedule[:-1]

# Step function for ODE flow. Choose second order "Heun" solver, s_churn = 0. gives deterministic
step_function = utils.get_step(order=2, s_churn=0.)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [41]:
word_pairs = [
    ("dog", "cat"),
    ("zebra", "horse"),
    ("bed", "table"),
    ("bear", "elephant"),
    ("airplane", "kite"),
    ("person", "monkey"),
    ("people", "monkeys"),
    ("teddy bear", "robot"),
    ("three", "no"),
    ("five", "no"),
]
word_swaps = {}  # Create an empty dictionary for word swaps
for a, b in word_pairs:
    word_swaps[a] = b
    word_swaps[b] = a
    word_swaps[a + 's'] = b + 's'
    word_swaps[b + 's'] = a + 's'

In [6]:
# TODO: change the data pipeline
# df = pd.read_csv('/root/tmp/EE142/data/coco/COCO-IT.csv')

In [34]:
# i = 0
# row = {
#     # 'Image ID': 397133,
#     "image_path": "/root/tmp/EE142/data/sk_test/five_bowls.png",
#     'Label': 'five',
#     'correct_obj+context': 'there are three bowls on the ground',
#     'context': 'there are bowls on the ground',
#     'correct_obj': 'three'
# }

# row = {
#     # 'Image ID': 397133,
#     "image_path": "/root/tmp/EE142/data/coco/val2017/000000577149.jpg",
#     'Label': 'zebras',
#     'correct_obj+context': 'there are four zebras walking on the side of this road',
#     'context': 'there are fout walking on the side of this road',
#     'correct_obj': 'zebras'
# }

# row = {
#     # 'Image ID': 397133,
#     "image_path": "/root/tmp/EE142/data/sk_test/Snipaste_2024-12-31_20-05-04.png",
#     'Label': 'monkey',
#     'correct_obj+context': 'The monkey with the hat is sitting on the ground',
#     'context': 'The monkey with the hat is sitting on the ground',
#     'correct_obj': 'monkey'
# }

row = {
    # 'Image ID': 397133,
    "image_path": "/root/tmp/EE142/data/coco/val2017/000000001296.jpg",
    'Label': 'lora',
    'correct_obj+context': 'figure of ',
    'context': 'The monkey with the hat is sitting on the ground',
    'correct_obj': 'monkey'
}



In [ ]:
# img_name = f"{row['Image ID']:012d}.jpg"

# path = os.path.join(data_in_dir, img_name)
path = row["image_path"]

img = Image.open(path).convert('RGB')
object, prompt = row['correct_obj'], row['correct_obj+context']
print(f"Processing {path} with object: {object} and prompt: {prompt}")

mod_prompt = utils.perform_word_swaps(prompt, {object: '_', object + 's': '_'})
word_swap = word_swaps.get(object, object)
swap_prompt = utils.perform_word_swaps(prompt, word_swaps)

print(f"""
    Original prompt: {prompt}
    Modified prompt: {mod_prompt}
    Word swap prompt: {swap_prompt}
""")

In [36]:
# Encode image to SD latent space
x_real_transformed = sdm.sdm_pipe.image_processor.preprocess(img, height=512, width=512).squeeze().permute((1, 2, 0))
x_real = sdm.encode_latents(img)

In [10]:
# plt.imshow(x_real_transformed)

In [ ]:
img_decoded = sdm.decode_latents(x_real)[0]
img_decoded
# img_numpy = img_decoded.sample.cpu().detach().numpy().squeeze().transpose(1, 2, 0)
# img_numpy = np.clip(img_numpy, 0, 1)

# Image.fromarray((img_numpy * 255).astype(np.uint8))

In [ ]:
# Encode prompts to CLIP embedding space

mod_prompt = "there are four _ walking on the side of this road"


print(f"""prompt = {prompt}
object = {object}
mod_prompt = {mod_prompt}
swap_prompt = {swap_prompt}
""")


v_org = sdm.encode_prompts(prompt).expand(batch_size, -1, -1)
v_null = sdm.encode_prompts('').expand(batch_size, -1, -1)
v_obj = sdm.encode_prompts(object).expand(batch_size, -1, -1)
v_mod = sdm.encode_prompts(mod_prompt).expand(batch_size, -1, -1)
v_swap = sdm.encode_prompts(swap_prompt).expand(batch_size, -1, -1)

In [ ]:
# Run in reverse to get the latent
latent_real = utils.reverse(sdm, step_function, schedule_reversible, x_real, prompt)

In [ ]:
img_decoded = sdm.decode_latents(latent_real)[0]
img_decoded

In [ ]:
# ######################################################
# ##                   No intervention                ##
# ######################################################
# # Then run forward (no intervention) and check recovery of real image - also track attention
# with t.cuda.amp.autocast(dtype=t.float16), t.no_grad():
#     with trace(sdm.sdm_pipe) as tc:
#         recover_real = utils.generate(sdm, step_function, schedule_reversible, latent_real, prompt)

# # Recover heat map
# heat_map = tc.compute_global_heat_map(prompt)
# heat_map = heat_map.compute_word_heat_map(object)  # keyword
# heat_map_lr = heat_map.value
# heat_map = F.interpolate(heat_map.value.unsqueeze(0).unsqueeze(0), size=(512, 512), mode='bilinear').squeeze(0).squeeze(0)
# # Decode real image without intervention
# recover_real_decode = sdm.decode_latents(recover_real)[0]
# recover_real_decode

In [ ]:
# plt.imshow(heat_map.cpu().detach().numpy())

In [ ]:
######################################################
##              Omit & Swap intervention            ##
######################################################
# Then run with a change in the prompt
print(f"Running with prompt: {mod_prompt}")
recover_mod = utils.generate(sdm, step_function, schedule_reversible, latent_real, mod_prompt)
recover_mod_decode = sdm.decode_latents(recover_mod)[0]

recover_mod_decode

In [ ]:
# Same, but swap a word instead
print(f"Running with prompt: {swap_prompt}")
recover_swap = utils.generate(sdm, step_function, schedule_reversible, latent_real, swap_prompt)
recover_swap_decode = sdm.decode_latents(recover_swap)[0]

recover_swap_decode

In [16]:
import gc 

gc.collect()
t.cuda.empty_cache()

In [ ]:
# Get pixel_nll
with t.cuda.amp.autocast(dtype=t.float16), t.no_grad():
    pixel_nll, pixel_mi_appx = itd.image_level_nll(x_real, v_org, 20, bs_=1)

In [ ]:
# Get Info heat maps
with t.cuda.amp.autocast(dtype=t.float16), t.no_grad():
    results_dict = itd.ll_ratio(x_real, v_null, v_obj, n_points=20,
                                n_samples_per_point=n_samples_per_point,
                                batch_size=batch_size, dim=(1,))

In [ ]:
mi_pixel_lr = results_dict['ll_ratio_pixel_app']
mi_pixel = F.interpolate(results_dict['ll_ratio_pixel_app'].unsqueeze(0).unsqueeze(0), size=(512, 512), mode='bilinear').squeeze(0).squeeze(0)
mi = results_dict['ll_ratio_pixel_app'].sum()

plt.imshow(mi_pixel.cpu().detach().numpy());
print(f"MI: {mi}")

## Word Phrase

In [ ]:
prompt = "there are four zebras walking on the side of this road"

word_list = ["four", "zebras", "walking", "road"]
results_list = []

for word in word_list:
    mod_prompt = prompt.replace(word, "_")
    print(f"Running with prompt: {mod_prompt}")

    v_mod = sdm.encode_prompts(mod_prompt).expand(batch_size, -1, -1)

    with t.cuda.amp.autocast(dtype=t.float16), t.no_grad():
        results_dict = itd.ll_ratio(x_real, v_mod, v_org, n_points=20,
                                    n_samples_per_point=n_samples_per_point,
                                    batch_size=batch_size, dim=(1,))
    cmi_pixel_lr = results_dict['ll_ratio_pixel_app']
    cmi_pixel = F.interpolate(results_dict['ll_ratio_pixel_app'].unsqueeze(0).unsqueeze(0), size=(512, 512), mode='bilinear').squeeze(0).squeeze(0)
    cmi = results_dict['ll_ratio_pixel'].sum()

    results_list.append(results_dict)

    # plt.imshow(cmi_pixel.cpu().detach().numpy());
    print(f"CMI: {cmi}")

In [ ]:
import sys
sys.path.append('/root/tmp/EE142/Info-Decomp')
from scripts.visual import *

# x_real_transformed = results_dict['x_real_transformed'].cpu()
thresh = 0.1

fig1, axs1 = plt.subplots(1, 1 + len(results_list), figsize=(3 * (1 + len(results_list)), 3))

plot_img(x_real_transformed, axs1[0], title='Real COCO image')

for i in range(len(results_list)):
    cmi_pixel_lr = results_list[i]['ll_ratio_pixel_app']
    cmi_pixel = F.interpolate(results_list[i]['ll_ratio_pixel_app'].unsqueeze(0).unsqueeze(0), size=(512, 512), mode='bilinear').squeeze(0).squeeze(0)
    cmi = results_list[i]['ll_ratio_pixel'].sum()

    plot_overlay(x_real_transformed, cmi_pixel, fig1, axs1[i+1], title=word_list[i] + ' $\mathfrak{i}(x;y|c)$',
                    normalize=False,
                    vmax=thresh, inset_text=cmi)
    
fig1.savefig("/root/tmp/EE142/data/sk_output/word/result.png")

In [ ]:
with t.cuda.amp.autocast(dtype=t.float16), t.no_grad():
    results_dict = itd.ll_ratio(x_real, v_swap, v_org, n_points=20,
                                n_samples_per_point=n_samples_per_point,
                                batch_size=batch_size, dim=(1,))
cmi_pixel_swap_lr = results_dict['ll_ratio_pixel_app']
cmi_pixel_swap = F.interpolate(results_dict['ll_ratio_pixel_app'].unsqueeze(0).unsqueeze(0), size=(512, 512), mode='bilinear').squeeze(0).squeeze(0)
cmi_swap = results_dict['ll_ratio_pixel'].sum()

plt.imshow(cmi_pixel_swap.cpu().detach().numpy());
print(f"CMI SWAP: {cmi_swap}")

## ITD

In [ ]:
img = Image.open(path).convert('RGB')
object, prompt = row['correct_obj'], row['correct_obj+context']
print(f"Processing {path} with object: {object} and prompt: {prompt}")

In [11]:
latent_images = sdm.encode_latents(img)
text_embeddings = sdm.encode_prompts("there are five bowls on the ground")
wro_embeddings = sdm.encode_prompts("there are three bowls on the ground")
uncond_embeddings = sdm.encode_prompts("")
conds = t.stack([text_embeddings, wro_embeddings, uncond_embeddings])

In [12]:
logsnrs = t.linspace(1.0 - 3.0 * 2.0, 1.0 + 3.0 * 2.0, 100).to(sdm.device)
z_sample_num = 1

In [ ]:
mmses, mmses_diff_appx = itd.image_level_mmse(latent_images, conds, logsnrs, total=z_sample_num, bs_=1)

In [ ]:
mmses_diff_appx.shape

In [ ]:
pixel_mmses, pixel_mmses_diff_appx = itd.pixel_level_mmse(latent_images, conds, logsnrs, total=z_sample_num, bs_=1)

In [25]:
pixel_mmses_up = t.zeros(list(pixel_mmses.shape[:-2]) + [512, 512])
pixel_mmses_diff_appx_up = t.zeros(list(pixel_mmses_diff_appx.shape[:-2]) + [512, 512])
for i in range(100):
    pixel_mmses_up[i] = F.interpolate(pixel_mmses[i], size=(512, 512), mode="bilinear")
    pixel_mmses_diff_appx_up[i] = F.interpolate(pixel_mmses_diff_appx[i], size=(512, 512), mode="bilinear")
pixel_mmses_up = pixel_mmses_up.permute(2, 1, 0, 3, 4)  # bs * 3 * snr_num * h * w
pixel_mmses_diff_appx_up = pixel_mmses_diff_appx_up.permute(2, 1, 0, 3, 4)  # bs * 3 * snr_num * h * w

In [ ]:
plt.imshow(pixel_mmses_up[0][2][0])

In [ ]:
# data_to_save = {
#     'mod_prompt': mod_prompt,
#     'object': object,
#     'word_swap': word_swap,
#     'x_real_transformed': x_real_transformed,
#     'recover_real_decode': recover_real_decode,
#     'recover_mod_decode': recover_mod_decode,
#     'recover_swap_decode': recover_swap_decode,
#     'recover_mod': recover_mod,
#     'recover_swap': recover_swap,
#     'recover_real': recover_real,
#     'cmi_pixel': cmi_pixel,
#     'cmi_pixel_swap': cmi_pixel_swap,
#     'mi_pixel': mi_pixel,
#     'cmi_pixel_lr': cmi_pixel_lr,
#     'cmi_pixel_swap_lr': cmi_pixel_swap_lr,
#     'mi_pixel_lr': mi_pixel_lr,
#     'heat_map_lr': heat_map_lr,
#     'heat_map': heat_map,
#     'cmi': cmi,
#     'cmi_swap': cmi_swap,
#     'mi': mi
# }

# # Save the dictionary to a file
# if not os.path.exists(res_out_dir):
#     os.makedirs(res_out_dir)
# t.save(data_to_save, os.path.join(res_out_dir, img_name[:-4] + '.pt'))

In [ ]:
# for i, row in df.iterrows():
#     img_name = f"{row['Image ID']:012d}.jpg"
#     path = os.path.join(data_in_dir, img_name)
#     img = Image.open(path).convert('RGB')
#     object, prompt = row['correct_obj'], row['correct_obj+context']
#     print(f"Processing {img_name} with object: {object} and prompt: {prompt}")

#     mod_prompt = utils.perform_word_swaps(prompt, {object: '_', object + 's': '_'})
#     word_swap = word_swaps.get(object, object)
#     swap_prompt = utils.perform_word_swaps(prompt, word_swaps)

#     # Encode image to SD latent space
#     x_real_transformed = sdm.sdm_pipe.image_processor.preprocess(img, height=512, width=512).squeeze().permute((1, 2, 0))
#     x_real = sdm.encode_latents(img)

#     # Encode prompts to CLIP embedding space
#     v_org = sdm.encode_prompts(prompt).expand(batch_size, -1, -1)
#     v_null = sdm.encode_prompts('').expand(batch_size, -1, -1)
#     v_obj = sdm.encode_prompts(object).expand(batch_size, -1, -1)
#     v_mod = sdm.encode_prompts(mod_prompt).expand(batch_size, -1, -1)
#     v_swap = sdm.encode_prompts(swap_prompt).expand(batch_size, -1, -1)

#     # Run in reverse to get the latent
#     latent_real = utils.reverse(sdm, step_function, schedule_reversible, x_real, prompt)
    
#     ######################################################
#     ##                   No intervention                ##
#     ######################################################
#     # Then run forward (no intervention) and check recovery of real image - also track attention
#     with t.cuda.amp.autocast(dtype=t.float16), t.no_grad():
#         with trace(sdm.sdm_pipe) as tc:
#             recover_real = utils.generate(sdm, step_function, schedule_reversible, latent_real, prompt)

#     # Recover heat map
#     heat_map = tc.compute_global_heat_map(prompt)
#     heat_map = heat_map.compute_word_heat_map(object)  # keyword
#     heat_map_lr = heat_map.value
#     heat_map = F.interpolate(heat_map.value.unsqueeze(0).unsqueeze(0), size=(512, 512), mode='bilinear').squeeze(0).squeeze(0)
#     # Decode real image without intervention
#     recover_real_decode = sdm.decode_latents(recover_real)[0]

#     ######################################################
#     ##              Omit & Swap intervention            ##
#     ######################################################
#     # Then run with a change in the prompt
#     recover_mod = utils.generate(sdm, step_function, schedule_reversible, latent_real, mod_prompt)
#     recover_mod_decode = sdm.decode_latents(recover_mod)[0]

#     # Same, but swap a word instead
#     recover_swap = utils.generate(sdm, step_function, schedule_reversible, latent_real, swap_prompt)
#     recover_swap_decode = sdm.decode_latents(recover_swap)[0]

#     # Get Info heat maps
#     with t.cuda.amp.autocast(dtype=t.float16), t.no_grad():
#         results_dict = itd.ll_ratio(x_real, v_null, v_obj, n_points=20,
#                                     n_samples_per_point=n_samples_per_point,
#                                     batch_size=batch_size, dim=(1,))
#     mi_pixel_lr = results_dict['ll_ratio_pixel_app']
#     mi_pixel = F.interpolate(results_dict['ll_ratio_pixel_app'].unsqueeze(0).unsqueeze(0), size=(512, 512), mode='bilinear').squeeze(0).squeeze(0)
#     mi = results_dict['ll_ratio_pixel'].sum()

#     with t.cuda.amp.autocast(dtype=t.float16), t.no_grad():
#         results_dict = itd.ll_ratio(x_real, v_mod, v_org, n_points=20,
#                                     n_samples_per_point=n_samples_per_point,
#                                     batch_size=batch_size, dim=(1,))
#     cmi_pixel_lr = results_dict['ll_ratio_pixel_app']
#     cmi_pixel = F.interpolate(results_dict['ll_ratio_pixel_app'].unsqueeze(0).unsqueeze(0), size=(512, 512), mode='bilinear').squeeze(0).squeeze(0)
#     cmi = results_dict['ll_ratio_pixel'].sum()

#     with t.cuda.amp.autocast(dtype=t.float16), t.no_grad():
#         results_dict = itd.ll_ratio(x_real, v_swap, v_org, n_points=20,
#                                     n_samples_per_point=n_samples_per_point,
#                                     batch_size=batch_size, dim=(1,))
#     cmi_pixel_swap_lr = results_dict['ll_ratio_pixel_app']
#     cmi_pixel_swap = F.interpolate(results_dict['ll_ratio_pixel_app'].unsqueeze(0).unsqueeze(0), size=(512, 512), mode='bilinear').squeeze(0).squeeze(0)
#     cmi_swap = results_dict['ll_ratio_pixel'].sum()

#     data_to_save = {
#         'mod_prompt': mod_prompt,
#         'object': object,
#         'word_swap': word_swap,
#         'x_real_transformed': x_real_transformed,
#         'recover_real_decode': recover_real_decode,
#         'recover_mod_decode': recover_mod_decode,
#         'recover_swap_decode': recover_swap_decode,
#         'recover_mod': recover_mod,
#         'recover_swap': recover_swap,
#         'recover_real': recover_real,
#         'cmi_pixel': cmi_pixel,
#         'cmi_pixel_swap': cmi_pixel_swap,
#         'mi_pixel': mi_pixel,
#         'cmi_pixel_lr': cmi_pixel_lr,
#         'cmi_pixel_swap_lr': cmi_pixel_swap_lr,
#         'mi_pixel_lr': mi_pixel_lr,
#         'heat_map_lr': heat_map_lr,
#         'heat_map': heat_map,
#         'cmi': cmi,
#         'cmi_swap': cmi_swap,
#         'mi': mi
#     }

#     # Save the dictionary to a file
#     if not os.path.exists(res_out_dir):
#         os.makedirs(res_out_dir)
#     t.save(data_to_save, os.path.join(res_out_dir, img_name[:-4] + '.pt'))